In [82]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import psycopg2
import time
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [83]:
password = open('password.txt','r').readline()
username = 'kmerrem'

#this is to write from geopandas to postgis table 
#gisdata_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/gisdata' %(username,password))
walkability_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/walkability' %(username,password))

In [84]:
cts = gpd.read_postgis("SELECT * FROM public.census_tracts_blank", walkability_engine, 'geom')

pednet = gpd.read_postgis("select * from final.pednet WHERE geom IS NOT NULL", walkability_engine, 'geom')
#pednet = gpd.read_postgis("select * from final.pednet_051519_z WHERE geom IS NOT NULL", walkability_engine, 'geom')

In [85]:
pednet.head(2)

,gid,road_type,sdwlk_code,sdwlk_desc,crosswalk,cwalk_type,px,px_type,geom
0,1,Local,7.0,Sidewalk on both sides,False,None,None,None,"(LINESTRING (299631.916 4843987.326, 299715.07..."
1,905,Collector,7.0,Sidewalk on both sides,False,None,None,None,"(LINESTRING (299303.7269 4832695.6224, 299336...."


In [86]:
pednet.dtypes

gid             int64
road_type      object
sdwlk_code    float64
sdwlk_desc     object
crosswalk      object
cwalk_type     object
px             object
px_type        object
geom           object
dtype: object

In [87]:
#jspatial joining of pednet to get ctnum on pednet segments
pednet_ct = gpd.sjoin(pednet,cts)


In [88]:
pednet_ct.head(2)

,gid_left,road_type,sdwlk_code,sdwlk_desc,crosswalk,cwalk_type,px,px_type,geom,index_right,gid_right,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,ctnum,geotext
0,1,Local,7.0,Sidewalk on both sides,False,None,None,None,"(LINESTRING (299631.916 4843987.326, 299715.07...",399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002
140,1569,Walkway,11.0,City walkway,True,ts,1921,ts,(LINESTRING (300095.3141264236 4843553.4388247...,399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002


In [89]:
#NEED THE INDEX TO RUN
cts =cts.set_index('ctnum')


In [90]:
cts.head(5)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom
ctnum,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,..."
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,(POLYGON ((314351.8668838083 4845056.100542194...
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,(POLYGON ((318379.9191213865 4837764.438619476...
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,"(POLYGON ((310234.199135254 4832619.302139398,..."


In [91]:
pednet_ct['sdwlk_length'] =np.nan
pednet_ct["sdwlk_length"]  =pednet_ct.length*1

# setting length times two when it is a local road and sidewalks code = 7 - both sides
idx = pednet_ct[(pednet_ct.sdwlk_code == 7.0) &(pednet_ct.road_type == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*2

# setting length to zero when it is a local road and sidewalks code = 3 - no sidewalk on either side
idx = pednet_ct[(pednet_ct.sdwlk_code == 3.0) &(pednet_ct.road_type == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*0

# setting length to zero when it is a local road and sidewalks code = 13 - not applicable
idx = pednet_ct[(pednet_ct.sdwlk_code == 13.0) &(pednet_ct.road_type == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*0

idx = pednet_ct[(pednet_ct.sdwlk_code == 13.0) &(pednet_ct.road_type == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*0


In [92]:
pednet_ct.head(5)

,gid_left,road_type,sdwlk_code,sdwlk_desc,crosswalk,cwalk_type,px,px_type,geom,index_right,gid_right,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,ctnum,geotext,sdwlk_length
0,1,Local,7.0,Sidewalk on both sides,False,None,None,None,"(LINESTRING (299631.916 4843987.326, 299715.07...",399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002,187.735360
140,1569,Walkway,11.0,City walkway,True,ts,1921,ts,(LINESTRING (300095.3141264236 4843553.4388247...,399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002,24.277944
175,1600,Local,5.0,Sidewalk on east side; partially on other side,False,None,None,None,(LINESTRING (300392.6727225842 4843590.4912553...,399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002,86.111947
567,2019,Local,7.0,Sidewalk on both sides,False,None,None,None,"(LINESTRING (300280.684 4844219.8229, 300319.1...",399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002,228.558020
2398,3913,Local,7.0,Sidewalk on both sides,False,None,None,None,"(LINESTRING (300044.4943 4844019.1848, 300149....",399,400,386,5350250.02,0250.02,35,Ontario,535,35535,Toronto,B,8455.230471,1.991021e+06,535025002.0,535025002,222.765748


In [93]:
#ratio_crosswalk = crosswalk segements = True / Total segments (True and False) e.g. 15 True / (15 True + 300 False) x 100 
for group, df in pednet_ct.groupby('ctnum'):
    cts.loc[group,"num_crosswalks"]  =df[df.crosswalk == True].crosswalk.count()
    cts.loc[group,"tot_crosswalk_leng"]  =df[df.crosswalk == True].length.sum()
    # make sure that null values in crosswalk_boolean are set to False 
    cts.loc[group,"ratio_crosswalk_leng"]  =df[df.crosswalk == True].length.sum()/df[df.crosswalk == False].length.sum()*100
    cts.loc[group,"ratio_crosswalks"]  =df[df.crosswalk == True].crosswalk.count()/df.crosswalk.count()*100
    cts.loc[group,"avg_crosswalk_leng"]  =df[df.crosswalk == True].length.mean()
    cts.loc[group,"med_crosswalk_leng"]  =df[df.crosswalk == True].length.median()
    cts.loc[group,"max_crosswalk_leng"]  =df[df.crosswalk == True].length.max()
    cts.loc[group,"sdwlk_length"]  =df.sdwlk_length.sum()

In [94]:
#cts

In [95]:
cts.head(5)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length
ctnum,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,46.0,1207.911260,2.798747,11.855670,26.258940,24.399220,50.590000,41951.544353
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25340.323861
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,(POLYGON ((314351.8668838083 4845056.100542194...,9.0,233.733311,0.958363,4.072398,25.970368,26.419112,31.194496,25165.054395
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,(POLYGON ((318379.9191213865 4837764.438619476...,11.0,202.243962,3.200885,17.741935,18.385815,18.104456,23.591011,10339.546070
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,"(POLYGON ((310234.199135254 4832619.302139398,...",0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,1275.167664


In [96]:
df[df.crosswalk == True].crosswalk.count()/df.crosswalk.count()*100

2.3904382470119523

In [97]:
df[df.crosswalk == True].length.sum()/df.length.sum()*100

0.5208422606904499

In [98]:
#crosswalk count
pednet_ct[pednet_ct.crosswalk == True].crosswalk.count()

12492

In [99]:
cts.head(5)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length
ctnum,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,46.0,1207.911260,2.798747,11.855670,26.258940,24.399220,50.590000,41951.544353
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25340.323861
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,(POLYGON ((314351.8668838083 4845056.100542194...,9.0,233.733311,0.958363,4.072398,25.970368,26.419112,31.194496,25165.054395
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,(POLYGON ((318379.9191213865 4837764.438619476...,11.0,202.243962,3.200885,17.741935,18.385815,18.104456,23.591011,10339.546070
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,"(POLYGON ((310234.199135254 4832619.302139398,...",0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,1275.167664


In [100]:
cts.sum(axis=0,skipna = True, numeric_only=None)

gid                                                                163878
__gid                                                              163878
ctuid                   5350001.005350002.005350264.005350082.00535000...
ctname                  0001.000002.000264.000082.000006.000163.000213...
pruid                   3535353535353535353535353535353535353535353535...
prname                  OntarioOntarioOntarioOntarioOntarioOntarioOnta...
cmauid                  5355355355355355355355355355355355355355355355...
cmapuid                 3553535535355353553535535355353553535535355353...
cmaname                 TorontoTorontoTorontoTorontoTorontoTorontoToro...
cmatype                 BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB...
shape_leng                                                    2.67813e+06
shape_area                                                    6.34184e+08
geotext                 5350001005350002005350264005350082005350006005...
num_crosswalks                        

In [101]:
#cts1 = cts1[cts1.geom.notna()]
#cts1

In [102]:
#You always need to drop the geometry column and write a new WKT text geometry column 

cts1 = cts.copy()
cts1 = cts1[cts1.geom.notna()]
cts1['geometry'] = cts1['geom']
cts1['geom'] = cts1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
cts1.drop('geometry', 1, inplace=True)
#Adds objectid as double. Need to change to integer
cts1['objectid'] = cts1.index.astype(np.int64)
cts1.to_sql('census_summary_cross', walkability_engine, schema='final', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})

In [103]:
cts1

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length,objectid
ctnum,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,MULTIPOLYGON (((320588.9306980086 4835855.6477...,46.0,1207.911260,2.798747,11.855670,26.258940,24.399220,50.590000,41951.544353,535000100
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,MULTIPOLYGON (((316642.831678931 4832470.55447...,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25340.323861,535000200
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,MULTIPOLYGON (((314351.8668838083 4845056.1005...,9.0,233.733311,0.958363,4.072398,25.970368,26.419112,31.194496,25165.054395,535026400
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,MULTIPOLYGON (((318379.9191213865 4837764.4386...,11.0,202.243962,3.200885,17.741935,18.385815,18.104456,23.591011,10339.546070,535008200
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,MULTIPOLYGON (((310234.199135254 4832619.30213...,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,1275.167664,535000600
535016300.0,6,217,5350163.00,0163.00,35,Ontario,535,35535,Toronto,B,2842.202163,4.540284e+05,535016300,MULTIPOLYGON (((309832.5952238032 4837976.0439...,7.0,113.803716,1.425627,10.769231,16.257674,16.863825,18.898599,13758.404982,535016300
535021302.0,7,443,5350213.02,0213.02,35,Ontario,535,35535,Toronto,B,14621.600658,7.041416e+06,535021302,MULTIPOLYGON (((302360.8525544122 4831898.5811...,113.0,3267.368976,6.865825,24.145299,28.914770,29.590508,51.861773,46065.485071,535021302
535037604.0,8,455,5350376.04,0376.04,35,Ontario,535,35535,Toronto,B,4439.479932,1.193620e+06,535037604,MULTIPOLYGON (((318932.6039103945 4852366.2186...,29.0,744.117349,4.043941,14.795918,25.659219,24.894897,33.370296,22742.406524,535037604
535024701.0,9,501,5350247.01,0247.01,35,Ontario,535,35535,Toronto,B,23043.297794,1.734167e+07,535024701,MULTIPOLYGON (((294567.331474375 4843841.86104...,114.0,3203.861442,3.169158,15.468114,28.104048,28.218350,45.639928,100445.532788,535024701
